In [1]:
%load_ext autoreload
%autoreload 2
import warnings 
warnings.filterwarnings("ignore")

In [2]:
from qiskit_metal import designs, MetalGUI
from qiskit_metal.qlibrary.sample_shapes.rectangle import Rectangle
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.Transmon_Interdigitated import TransmonInterdigitated
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.designs.design_multiplanar import MultiPlanar
import pandas as pd


# Main Design

In [3]:
design = MultiPlanar({},overwrite_enabled = True, layer_stack_filename = "layer_stack.txt")

gui = MetalGUI(design)

## Qubit Design

In [4]:
design.delete_all_components()

#### Design so Resonator is coupled strongly to each qubit-> large cap leads to large dispersive shift

conn_pads1 = dict(connection_pads = dict(coupler = dict(loc_W=1, loc_H=1, pad_width = '250um'),
                                         readout = dict(loc_W=-1, loc_H=-1, pad_width = '250um')))

conn_pads2 = dict(pad_width = '550um',
                  connection_pads = dict(coupler = dict(loc_W = -1, loc_H = 1,pad_width = '275um'),
                                         readout = dict(loc_W=1, loc_H=-1, pad_width='275um')))
q1 = TransmonPocket(design, "Q1", options=dict(pos_x='-0.5mm', pos_y='+0.0mm', layer=1, **conn_pads1))
q2 = TransmonPocket(design, "Q2", options=dict(pos_x='+0.5mm', pos_y='+0.0mm', layer=1, **conn_pads2))

gui.rebuild()
gui.autoscale()

## Coupler Design

In [5]:
coupler_options = dict(pin_inputs = dict(start_pin = dict(component=q2.name, pin='coupler'),
                                         end_pin = dict(component=q1.name, pin='coupler')))

coupler_connector = RouteStraight(design, 'coupler_connector', options = coupler_options)

gui.rebuild()
gui.autoscale()

## Readout Resonator Design

In [6]:
otg11_options = dict(pos_x='-1.5mm', pos_y = '0.0mm', orientation='180', layer='3')
otg21_options = dict(pos_x='+1.5mm', pos_y = '0.0mm', orientation='0', layer='3')
otg12_options = dict(pos_x='-0.955mm', pos_y = '-0.195mm', orientation='0', layer='3')
otg22_options = dict(pos_x='+0.955mm', pos_y = '-0.1955mm', orientation='180', layer='3')
otg11 = OpenToGround(design, 'otg11' ,options=otg11_options)
otg21 = OpenToGround(design, 'otg21' ,options=otg21_options)
otg12 = OpenToGround(design, 'otg12' ,options=otg12_options)
otg22 = OpenToGround(design, 'otg22' ,options=otg22_options)

readout1_options = dict(total_length = '5.97mm', fillet = '40um', 
                        pin_inputs = dict(start_pin = dict(component = otg11.name, pin='open'),
                                          end_pin = dict(component = otg12.name, pin = 'open')),
                        lead = dict(start_straight='150um'),
                        meander=dict(spacing = '100um', asymmetry = '0'),
                        layer = '3')

readout2_options = dict(total_length = '5.97mm', fillet = '40um', 
                        pin_inputs = dict(start_pin = dict(component = otg22.name, pin='open'),
                                          end_pin = dict(component = otg21.name, pin = 'open')),
                        lead = dict(start_straight='120um'),
                        meander=dict(spacing = '100um', asymmetry = '0.2'),
                        layer = '3')

readout1 = RouteMeander(design, 'readout1', options = readout1_options)
readout2 = RouteMeander(design, 'readout2', options = readout2_options)

gui.rebuild()
gui.autoscale()

## Via Creation and Dielectric

In [7]:
via_positions = [('-0.94mm','-0.1950mm'),('+0.94mm','-0.1950mm')]

for layer in ['1','2','3']:
    for via, positions in enumerate(via_positions):
        for subtract in zip([False,True],['30um','42um']):
            via_size = '20um' if layer =='2' else subtract[1]
            actual_layer = '5' if (layer == '2' and subtract[0]) else layer
            via_options = dict(width = via_size,
                               height = via_size,
                               pos_x = positions[0],
                               pos_y = positions[1],
                               subtract = subtract[0],
                               layer = actual_layer,
                               orientation = '0',
                               helper = 'False',
                               chip = 'main' )
            name = 'via' +str(via+1) +'_layer' + actual_layer+('' if not subtract[0] else '_sub')
            Rectangle(design, name, via_options)
            #print(subtract)
            #print(layer == '2' and subtract[0])
    
gui.rebuild()
gui.autoscale()
            

# Render and Extract Capacitance Matrix

In [8]:
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import QElmerRenderer

## Render

In [9]:
#elmer_renderer = QElmerRenderer(design, layer_types = dict(metal = [1,2,3], dielectric = [4,5]))
#elmer_renderer.render_design(mesh_geoms = True,
#                             skip_junctions= True,
#                             open_pins=[("Q1","readout"),("Q2","readout")],
#                             omit_ground_for_layers = [2])

In [10]:
#elmer_renderer.launch_gmsh_gui()

## Generate Capacitance Matrix

In [11]:
#elmer_renderer.export_mesh()
#elmer_renderer.add_solution_setup('capacitance')
#elmer_renderer.run('capacitance')

In [12]:
#cap_matrix = elmer_renderer.capacitance_matrix
#cap_matrix

In [13]:
#elmer_renderer.display_post_processing_data()
#elmer_renderer.close()

In [14]:
elmer_renderer = QElmerRenderer(design, layer_types=dict(metal=[1,2,3], dielectric=[4,5]))
select = ['Q1','via1_layer1', 'via1_layer2', 'via1_layer3', 'via1_layer1_sub', 'via1_layer5_sub', 'via1_layer3_sub']
#select = ['Q1']
elmer_renderer.render_design(selection=select,  open_pins=[("Q1","readout"),("Q1","coupler")], skip_junctions=True, 
                            omit_ground_for_layers = [2])

#elmer_renderer.launch_gmsh_gui()

08:04PM 49s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png

08:04PM 49s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png



In [15]:
elmer_renderer.export_mesh()
elmer_renderer.add_solution_setup('capacitance')
elmer_renderer.run('capacitance')
cap_matrix_q1 = elmer_renderer.capacitance_matrix
cap_matrix_q1.to_csv('cap_matrix_q1.csv')
cap_matrix_q1


08:05PM 10s INFO [run]: Running ElmerGrid on input mesh from Gmsh...
08:05PM 11s INFO [run]: Running ElmerSolver for solver type: 'capacitance'


,Q1_coupler_connector_pad,via1_layer3_rectangle,Q1_pad_top,Q1_pad_bot,ground_plane
Q1_coupler_connector_pad,163.631085,-0.338135,-42.562698,-2.725724,-118.004528
via1_layer3_rectangle,-0.338135,215.679540,-2.739835,-42.566531,-170.035039
Q1_pad_top,-42.562698,-2.739835,250.126084,-50.567075,-154.256476
Q1_pad_bot,-2.725724,-42.566531,-50.567075,250.075780,-154.216449
ground_plane,-118.004528,-170.035039,-154.256476,-154.216449,300.000000


In [16]:
elmer_renderer = QElmerRenderer(design, layer_types=dict(metal=[1,2,3], dielectric=[4,5]))
select = ['Q2','via2_layer1', 'via2_layer2', 'via2_layer3', 'via2_layer1_sub', 'via2_layer5_sub', 'via2_layer3_sub']
#select = ['Q2']
elmer_renderer.render_design(selection=select,  open_pins=[("Q2","readout"),("Q2","coupler")], skip_junctions=True, 
                            omit_ground_for_layers = [2])

#elmer_renderer.launch_gmsh_gui()

In [17]:
elmer_renderer.export_mesh()
elmer_renderer.add_solution_setup('capacitance')
elmer_renderer.run('capacitance')
cap_matrix_q2 = elmer_renderer.capacitance_matrix
cap_matrix_q2.to_csv('cap_matrix_q2.csv')
cap_matrix_q2


08:05PM 46s INFO [run]: Running ElmerGrid on input mesh from Gmsh...
08:05PM 47s INFO [run]: Running ElmerSolver for solver type: 'capacitance'


,Q2_coupler_connector_pad,via2_layer3_rectangle,Q2_pad_top,Q2_pad_bot,ground_plane
Q2_coupler_connector_pad,168.277993,-0.270188,-45.617732,-2.710089,-119.679984
via2_layer3_rectangle,-0.270188,220.584424,-2.732696,-45.566898,-172.014641
Q2_pad_top,-45.617732,-2.732696,296.128120,-60.479065,-187.298627
Q2_pad_bot,-2.710089,-45.566898,-60.479065,296.211562,-187.455510
ground_plane,-119.679984,-172.014641,-187.298627,-187.455510,300.000000


In [18]:
c2 = pd.read_csv('cap_matrix_q2.csv').set_index('Unnamed: 0',True)
c2.index.name = None
c2

,Q2_coupler_connector_pad,via2_layer3_rectangle,Q2_pad_top,Q2_pad_bot,ground_plane
Q2_coupler_connector_pad,168.277993,-0.270188,-45.617732,-2.710089,-119.679984
via2_layer3_rectangle,-0.270188,220.584424,-2.732696,-45.566898,-172.014641
Q2_pad_top,-45.617732,-2.732696,296.128120,-60.479065,-187.298627
Q2_pad_bot,-2.710089,-45.566898,-60.479065,296.211562,-187.455510
ground_plane,-119.679984,-172.014641,-187.298627,-187.455510,300.000000


# LOM Analysis

In [21]:
import scqubits as scq
from scipy.constants import speed_of_light as c_light
import matplotlib.pyplot as plt
from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem, QuantumSystemRegistry
import pandas as pd
cap_matrix_q1 = pd.read_csv('cap_matrix_q1.csv').set_index('Unnamed: 0',True)
cap_matrix_q1.index.name = None

cap_matrix_q2 = pd.read_csv('cap_matrix_q2.csv').set_index('Unnamed: 0',True)
cap_matrix_q2.index.name = None


## LOM Cells

In [22]:
opt1 = dict(node_rename = {'Q1_coupler_connector_pad':'coupling','via1_layer3_rectangle': 'readout_alice'},
#opt1 = dict(node_rename = {'Q1_coupler_connector_pad':'coupling','Q1_readout_connector_pad': 'readout_alice'},
            cap_mat = cap_matrix_q1,
            ind_dict = {('Q1_pad_top','Q1_pad_bot'):10},
            jj_dict = {('Q1_pad_top','Q1_pad_bot'):'j1'},
            cj_dict = {('Q1_pad_top','Q1_pad_bot'):2}
           )            

cell_1 = Cell(opt1)

opt2 = dict(node_rename = {'Q2_coupler_connector_pad':'coupling','via2_layer3_rectangle': 'readout_bob'},
#opt2 = dict(node_rename = {'Q2_coupler_connector_pad':'coupling','Q2_readout_connector_pad': 'readout_bob'},
            cap_mat = cap_matrix_q2,
            ind_dict = {('Q2_pad_top','Q2_pad_bot'):12},
            jj_dict = {('Q2_pad_top','Q2_pad_bot'):'j2'},
            cj_dict = {('Q2_pad_top','Q2_pad_bot'):2}
           )
cell_2 = Cell(opt2)

## Subsystems

In [23]:
transmon_alice = Subsystem(name = 'transmon_alice', sys_type='TRANSMON', nodes=['j1'])

transmon_bob = Subsystem(name = 'transmon_bob', sys_type='TRANSMON', nodes=['j2'])


### Readout resonator properties

#Alice
q_opts1 = dict(
    f_res = 5.6, ##in GHz
    Z0 = 50,  #charicterisit impedance
    vp = 0.40314 * c_light, ## phase velocity
)
res_alice = Subsystem(name='readout_alice', sys_type ='TL_RESONATOR', nodes = ['readout_alice'], q_opts = q_opts1)


#Bob
q_opts2 = dict(
    f_res = 4.0, ##in GHz
    Z0 = 50,  #charicterisit impedance
    vp = 0.40314 * c_light, ## phase velocity
)
res_bob = Subsystem(name='readout_bob', sys_type ='TL_RESONATOR', nodes = ['readout_bob'], q_opts = q_opts2)






## Create Composite System

In [24]:
composite_sys = CompositeSystem(
            subsystems = [transmon_alice, transmon_bob, res_alice, res_bob],
            cells = [cell_1, cell_2],
            grd_node = 'ground_plane', #ID Ground plane to remove
            nodes_force_keep = ['readout_alice', 'readout_bob'] #need to do this so hamiltonian includes readout res's
            )

In [25]:
cg = composite_sys.circuitGraph()
print(cg)

node_jj_basis:
-------------

['j1', 'Q1_pad_bot', 'j2', 'Q2_pad_bot', 'readout_alice', 'readout_bob', 'coupling']

nodes_keep:
-------------

['j1', 'j2', 'readout_alice', 'readout_bob']


L_inv_k (reduced inverse inductance matrix):
-------------

                j1        j2  readout_alice  readout_bob
j1             0.1  0.000000            0.0          0.0
j2             0.0  0.083333            0.0          0.0
readout_alice  0.0  0.000000            0.0          0.0
readout_bob    0.0  0.000000            0.0          0.0

C_k (reduced capacitance matrix):
-------------

                       j1          j2  readout_alice  readout_bob
j1             151.101515   -1.327943      19.577087    -0.323168
j2              -1.327943  178.893657      -0.365381    21.064629
readout_alice   19.577087   -0.365381     210.442577    -0.088919
readout_bob     -0.323168   21.064629      -0.088919   215.550721




In [26]:
hilbertspace = composite_sys.create_hilbertspace()
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

Quantum object: dims = [[10, 10, 3, 3], [10, 10, 3, 3]], shape = (900, 900), type = oper, isherm = True
Qobj data =
[[-26241.52718463+0.j              0.        +0.05621421j
       0.        +0.j         ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 [     0.        -0.05621421j -22241.52718463+0.j
       0.        +0.0794989j  ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 [     0.        +0.j              0.        -0.0794989j
  -18241.52718463+0.j         ...      0.        +0.j
       0.        +0.j              0.        +0.j        ]
 ...
 [     0.        +0.j              0.        +0.j
       0.        +0.j         ...  39669.84020715+0.j
       0.        +1.02813946j      0.        +0.j        ]
 [     0.        +0.j              0.        +0.j
       0.        +0.j         ...      0.        -1.02813946j
   43669.84020715+0.j              0.        +1.45400877j]
 [     0.        +0.j              0.    

In [27]:
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

Transmon------------| [Transmon_3]
                    | EJ: 16346.15128067812
                    | EC: 129.7656867220069
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Transmon------------| [Transmon_4]
                    | EJ: 13621.792733898432
                    | EC: 109.54535603906555
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Oscillator----------| [Oscillator_3]
                    | E_osc: 5600.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_4]
                    | E_osc: 4000.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                  

In [28]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count = 30) # anharminicity are the diagonal in MHz
#off diagonal show dispersive shifts


system frequencies in GHz:
--------------------------
{'transmon_alice': 3.9754412774082737, 'transmon_bob': 3.331395800274382, 'readout_alice': 5.607133021117741, 'readout_bob': 4.0083819587573455}

Chi matrices in MHz
--------------------------
                transmon_alice  transmon_bob  readout_alice  readout_bob
transmon_alice     -138.396851     -0.239787      -1.779944    -0.028931
transmon_bob         -0.239787   -115.285370      -0.000678    -3.157666
readout_alice        -1.779944     -0.000678       4.283129    -0.000973
readout_bob          -0.028931     -3.157666      -0.000973     2.390156


In [29]:
transmon_alice.h_params

defaultdict(dict,
            {'j1': {'EJ': 16346.15128067812,
              'EC': 129.7656867220069,
              'Q_zpf': 3.204353268e-19,
              'default_charge_op': Operator(op=array([[-22,   0,   0, ...,   0,   0,   0],
                     [  0, -21,   0, ...,   0,   0,   0],
                     [  0,   0, -20, ...,   0,   0,   0],
                     ...,
                     [  0,   0,   0, ...,  20,   0,   0],
                     [  0,   0,   0, ...,   0,  21,   0],
                     [  0,   0,   0, ...,   0,   0,  22]]), add_hc=False)}})

In [30]:
transmon_bob.h_params

defaultdict(dict,
            {'j2': {'EJ': 13621.792733898432,
              'EC': 109.54535603906555,
              'Q_zpf': 3.204353268e-19,
              'default_charge_op': Operator(op=array([[-22,   0,   0, ...,   0,   0,   0],
                     [  0, -21,   0, ...,   0,   0,   0],
                     [  0,   0, -20, ...,   0,   0,   0],
                     ...,
                     [  0,   0,   0, ...,  20,   0,   0],
                     [  0,   0,   0, ...,   0,  21,   0],
                     [  0,   0,   0, ...,   0,   0,  22]]), add_hc=False)}})

# Time Evolution Simulation (Sequencing)

In [31]:
from tqdm import tqdm
from functools import partial
import qutip

from sequencing import Qubit, Transmon, Cavity, System, get_sequence, sync, delay, QasmSequence
from sequencing.calibration import fit_sine, fit_displacement, tune_rabi, tune_displacement, tune_drag

from qiskit_metal.analyses.quantization.lom_time_evolution_sim import lom_composite_sys_to_seq_sys

In [32]:
system = lom_composite_sys_to_seq_sys(composite_sys, hilbertspace, levels = [3,3,10,10])


In [34]:
system.modes #Mode is a subsystem in LOM analysis

[Transmon(name='transmon_bob', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.11528537010490254, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, length=100, sigma=0, shape='tanh'), 'gaussian_pulse': GaussianPulse(name='gaussian_pulse', cls='sequencing.pulses.GaussianPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, sigma=10, chop=4, drag=0.0)}, default_pulse='gaussian_pulse'),
 Transmon(name='transmon_alice', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.13839685059551993, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0

NameError: name 'selective_sigma' is not defined

## Simulate Alice

In [36]:
alice = system.modes[1]
readout_alice = system.modes[3]
print(alice)
print(readout_alice)

Transmon(name='transmon_alice', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.13839685059551993, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, length=100, sigma=0, shape='tanh'), 'gaussian_pulse': GaussianPulse(name='gaussian_pulse', cls='sequencing.pulses.GaussianPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, sigma=10, chop=4, drag=0.0)}, default_pulse='gaussian_pulse')
Cavity(name='readout_alice', cls='sequencing.modes.Cavity', levels=10, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=0.004283129069925053, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noi

In [39]:
selective_sigma = 100 #ns

with system.use_modes([alice]):
        with alice.temporarily_set(gaussian_pulse__sigma = selective_sigma):
            _, _, selective_qubit_amp = tune_rabi(
            system, system.fock(transmon_alice = 0, transmon_bob = 0, readout_alice =0, readout_bob=0), mode_name = alice.name,
            update = False, verify=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 90.27it/s]


In [40]:
def selective_rotation(qubit, angle, phase =0, detune=0, sigma = selective_sigma):
    with qubit.gaussian_pulse.temporarily_set(sigma=sigma, amp  = selective_qubit_amp):
        qubit.rotate(np.pi, phase, detune=detune)

init_states = [
    (f'$|g{n}\\rangle$', system.fock(transmon_alice=0, readout_alice=n)) for n in range(4)
]

In [43]:
import numpy as np
results = {}

seq = get_sequence(system)
selective_rotation(alice, np.pi)

for label, state in tqdm(init_states, desc='Initial states'):
    result = seq.run(state)
    results[label] = result

Initial states: 100%|████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  8.26it/s]


In [44]:
fig, ax = plt.subplots(1,1)
for label, result in results.items():
    qubit_states = [state.ptrace(alice.index) for state in result.states]
    e_pops = qutip.expect(alice.fock_dm(1, full_space=False), qubit_states)
    ax.plot(result.times, e_pops, label=label)
    ax.grid(True)
    ax.legend(loc=0)
ax.set_xlabel('Times [ns]')
ax.set_ylabel(r'$P(|e\rangle)$')
_ = ax.set_title('Transmon trajectory vs. Initital cavity state')

In [45]:
bob = system.modes[0]
readout_bob = system.modes[2]
print(bob)
print(readout_bob)

Transmon(name='transmon_bob', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.11528537010490254, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, length=100, sigma=0, shape='tanh'), 'gaussian_pulse': GaussianPulse(name='gaussian_pulse', cls='sequencing.pulses.GaussianPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, sigma=10, chop=4, drag=0.0)}, default_pulse='gaussian_pulse')
Cavity(name='readout_bob', cls='sequencing.modes.Cavity', levels=10, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=0.0023901562353894406, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_

In [54]:
selective_sigma = 300 #ns

with system.use_modes([bob]):
        with bob.temporarily_set(gaussian_pulse__sigma = selective_sigma):
            _, _, selective_qubit_amp = tune_rabi(
            system, system.fock(transmon_alice = 0, transmon_bob = 0, readout_alice =0, readout_bob=0), mode_name = bob.name,
            update = False, verify=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:01<00:00, 37.40it/s]


In [55]:
init_states = [
    (f'$|g{n}\\rangle$', system.fock(transmon_bob=0, readout_bob=n)) for n in range(4)
]

In [56]:
import numpy as np
results = {}

seq = get_sequence(system)
selective_rotation(bob, np.pi)

for label, state in tqdm(init_states, desc='Initial states'):
    result = seq.run(state)
    results[label] = result

Initial states: 100%|████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  7.46it/s]


In [57]:
fig, ax = plt.subplots(1,1)
for label, result in results.items():
    qubit_states = [state.ptrace(bob.index) for state in result.states]
    e_pops = qutip.expect(bob.fock_dm(1, full_space=False), qubit_states)
    ax.plot(result.times, e_pops, label=label)
    ax.grid(True)
    ax.legend(loc=0)
ax.set_xlabel('Times [ns]')
ax.set_ylabel(r'$P(|e\rangle)$')
_ = ax.set_title('Transmon trajectory vs. Initital cavity state')

In [100]:
Qubits  = [alice,bob]
qreg = System('qreg',modes = Qubits)
n=len(Qubits)
qreg

System(name='qreg', cls='sequencing.system.System', modes=[Transmon(name='transmon_bob', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.11802665523139877, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, length=100, sigma=0, shape='tanh'), 'gaussian_pulse': GaussianPulse(name='gaussian_pulse', cls='sequencing.pulses.GaussianPulse', amp=1.0, detune=0.0, phase=0.0, dt=1, noise_sigma=0.0, noise_alpha=0.0, scale_noise=False, sigma=10, chop=4, drag=0.0)}, default_pulse='gaussian_pulse'), Transmon(name='transmon_alice', cls='sequencing.modes.Transmon', levels=3, t1=inf, t2=inf, thermal_population=0.0, df=0.0, kerr=-0.14079800368877476, pulses={'smoothed_constant_pulse': SmoothedConstantPulse(name='smoothed_constant_pulse', cls='sequencing.pulses.SmoothedConstantPuls

In [87]:
for qubit in Qubits:
    with system.use_modes([qubit]):
        with qubit.temporarily_set(gaussian_pulse__sigma = selective_sigma):
            _, _, selective_qubit_amp = tune_rabi(
            system, system.fock(transmon_alice = 0, transmon_bob = 0, readout_alice =0, readout_bob=0), mode_name = qubit.name,
            update = False, verify=False)

100%|█████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 220.56it/s]


In [93]:
QASM_CIRCUIT = [
    f'OPENQASM 2.0;',
    f'include "qelib1.inc";',
    f'qreg q[{n}];',
    f'h q[0];',
    f'barrier;',
]
QASM_CIRCUIT.extend(
    [f'CX q[0],q[{i}];' for i in range(1,n)]
)

QASM_CIRCUIT = '\n\t'.join([''] + QASM_CIRCUIT)

print('Running the following QASM circuit:')
print(QASM_CIRCUIT)

seq = QasmSequence(system)
seq.qasm_circuit(QASM_CIRCUIT, unitary=False, append=True)

Running the following QASM circuit:

	OPENQASM 2.0;
	include "qelib1.inc";
	qreg q[2];
	h q[0];
	barrier;
	CX q[0],q[1];


AttributeError: If unitary is False, all qubits must implement rotate_y (readout_alice does not).

In [102]:
ax = qubit.gaussian_pulse.plot()
_ = ax.set_xlabel('Time [ns]')